## MLP-mixer _convert 


In [5]:
import torch
import numpy as np
from erc.model import MLPMixer

audio_output = torch.ones([3, 1024]) # Batch, Seq
text_output = torch.ones([3, 768]) # Batch, Seq


# Performs a batch matrix-matrix product of matrices stored in input and mat2.
matmul_output = torch.bmm(audio_output.unsqueeze_(2), text_output.unsqueeze_(1))
print(matmul_output.unsqueeze_(1).shape) # Batch, Color, Width, Hight


model = MLPMixer(
    image_size = (1024, 768),
    channels = 1,
    patch_size = 16,
    dim = 512,
    depth = 12,
    num_classes = 7
)
pred = model(matmul_output) 
print(pred.shape)



## AI-Hub dataset


In [6]:
import os 
from glob import glob

def get_hub_txt(self, txt: str, encoding: str = None)-> Tuple[torch.Tensor, torch.Tensor]:
    if self.tokenizer:
        result: dict = self.tokenizer(text=txt,
                                    padding="max_length",
                                    truncation="only_first",
                                    max_length=self.max_length_txt,
                                    return_attention_mask=True,
                                    return_tensors="pt")
        input_ids = result["input_ids"].squeeze()
        mask = result["attention_mask"].squeeze()
        return input_ids, mask
    else:
        return txt, None
    
class AIHubDialog():
    # PRETRAINED_DATA_PATH = '/home/hoesungryu/workspace/AI-Hub_emotion_dialog'
    def __init__(self, PRETRAINED_DATA_PATH):
        self.txt_folder = sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'annotation')+'/*.csv'))
        self.wav_folder = sorted(glob(os.path.join(PRETRAINED_DATA_PATH,'wav')+'/*.wav'))

    def __len__(self):
        assert len(glob(self.wav_folder)) == len(glob(self.txt_folder))
        return len(glob(self.wav_folder)) 
    
    def __getitem__(self, idx:int):
        data = {}
        txt, _, emotion = pd.read_csv(self.txt_folder[idx]).iloc[0].values
    
        # Txt File
        txt, txt_mask = self.get_hub_txt(txt_path=txt, encoding=self.TEXT_ENCODING)
        data["txt"] = txt
        data["txt_mask"] = txt_mask
    
        # emotion 
        data["emotion"] = self.get_emo(emotion)

        sampling_rate, wav, wav_mask = self.get_wav(wav_path=self.wav_folder[idx])
        data["sampling_rate"] = sampling_rate
        data["wav"] = wav
        data["wav_mask"] = wav_mask

        return data
    
import random 
random.seed(42)

@staticmethod
def sampling_with_ratio(total_len : int, train_ratio = 0.8):
    total_len = wav_folder 
    total_idx = [i for i in range(total_len)]
    train_num = int(total_len * train_ratio)

    train_idx = random.sample(total_idx, train_num)
    valid_idx = list(set(total_idx) - set(train_idx))

    return train_idx, valid_idx

@staticmethod
def get_multiple_elements_in_list(in_list, in_indices):
    """리스트에서 복수인덱스 값을 가져온다"""
    return [in_list[i] for i in in_indices]

torch.Size([3, 7])


## Weighted Cross-entropy


In [20]:
# Get Centroid ... 
from collections import defaultdict

emotion_va_dic = defaultdict(dict)

emotion_va_dic[emotion+'_arousal'] = arousal
emotion_va_dic[emotion+'_valence'] = valence



In [ ]:
for emotion_idx in range(0,7):
    

In [21]:
import logging
import hydra

import erc
from torch.utils.data import DataLoader
from datasets import load_from_disk

train_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_train4")
valid_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_valid4")


train_dataloadaer = DataLoader(train_dataset, batch_size=2)
sample = next(iter(train_dataloadaer))

In [24]:
train_dataset[train_dataset['emotion'] ==3]

{'segment_id': ['Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_F001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001',
  'Sess01_script01_M001'

In [25]:
train_dataset['emotion'].numpy()

array([0, 1, 2, ..., 3, 3, 3])

In [37]:

class_count = Counter(train_dataset['emotion'].detach().cpu().numpy())
class_count

Counter({0: 898, 1: 365, 2: 1564, 3: 11586, 4: 2088, 5: 701, 6: 308})

In [31]:
nSamples = class_count.values()
nSamples

dict_values([898, 365, 1564, 11586, 2088, 701, 308])

In [35]:
import torch
# nSamples = [887, 6130, 480, 317, 972, 101, 128]
normedWeights = torch.FloatTensor([1 - (x / sum(nSamples)) for x in nSamples])
# normedWeights = torch.FloatTensor(normedWeights)

loss = nn.CrossEntropyLoss(normedWeights)


In [40]:
import logging
import hydra
import torch.nn as  nn 
import erc
from torch.utils.data import DataLoader
from datasets import load_from_disk

train_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_train4")
valid_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_valid4")


from collections import Counter

def get_classweights(traindataset)-> torch.FloatTensor:
    class_count = Counter(traindataset['emotion'].numpy())
    nSamples = class_count.values()
    normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
    return torch.FloatTensor(normedWeights)

train_normedWeight = get_classweights(train_dataset)
valid_normedWeight = get_classweights(valid_dataset)


In [41]:
# loss = nn.CrossEntropyLoss(train_normedWeight)
train_normedWeight


tensor([0.9487, 0.9792, 0.9107, 0.3383, 0.8808, 0.9600, 0.9824])

## Get Ground Truth emotion in Multi-label-class

```python 

logits = self.mlp_mixer(matmul_output) # (B, num_labels)

# calcuate the loss fct
cls_logits = logits[:, :-2]
cls_labels = labels["emotion"]
if cls_labels.ndim == 1:
    # Single label case
    cls_loss = self.criterions["cls"](cls_logits, cls_labels.long())
elif cls_labels.ndim == 2:
    # Multi label case
    cls_loss = self.criterions["cls"](cls_logits, cls_labels.float())
    cls_labels = labels["emotion_deuce"] # trainer 도 바꿔줘야됨 ... 

reg_logits = logits[:, -2:]
reg_loss = self.criterions["reg"](reg_logits, labels["regress"].float())

total_loss = cls_loss * self.cls_coef + reg_loss * self.reg_coef
return {
    "loss": total_loss,
    "cls_loss": cls_loss.detach().cpu(),
    "reg_loss": reg_loss.detach().cpu(),
    "emotion": cls_labels.detach(),
    "regress": labels["regress"].detach(),
    "cls_pred": cls_logits.detach(),
    "reg_pred": reg_logits.detach(),
}

```

In [190]:
# get centroid 
import logging
import hydra
import pandas as pd


import torch.nn as  nn 

from torch.utils.data import DataLoader
from datasets import load_from_disk

import erc

train_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_train4")
valid_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19-kemdy20_valid4")


def dataset2df(dataset, columns = ['emotion','valence','arousal']):
    df = pd.DataFrame()
    for column in columns:
        df[column] = dataset[column]
    return df

train_df = dataset2df(train_dataset)
valid_df = dataset2df(valid_dataset)

total_df = pd.concat([train_df,valid_df], axis=0)
display(total_df)
# Get Centroid ... 
from collections import defaultdict

emotion_va_dict = defaultdict(dict)

for emotion_idx in range(0,7):
    valence = total_df[total_df['emotion']==emotion_idx]['valence'].mean()
    arousal = total_df[total_df['emotion']==emotion_idx]['arousal'].mean()
    # print()
    emotion_va_dict[f'{emotion_idx}_centroid'] = (valence, arousal)
emotion_va_dict

,emotion,valence,arousal
0,0,1.7,4.0
1,1,1.5,3.6
2,2,1.3,4.3
3,2,1.5,3.8
4,1,1.6,3.7
...,...,...,...
4673,3,3.0,3.0
4674,0,2.7,3.5
4675,3,2.9,3.5
4676,3,2.3,3.5


defaultdict(dict,
            {'0_centroid': (2.7013428, 3.6017902),
             '1_centroid': (1.8056872, 3.5810425),
             '2_centroid': (1.8337096, 3.7831194),
             '3_centroid': (2.989782, 3.1387265),
             '4_centroid': (4.093699, 3.7420354),
             '5_centroid': (1.9814088, 2.6549654),
             '6_centroid': (2.2050884, 3.1471236)})

In [243]:
# get centroid 
import logging
import hydra
import pandas as pd


import torch.nn as  nn 

from torch.utils.data import DataLoader
from datasets import load_from_disk

import erc

train_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19_train4")
valid_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy19_valid4")


def dataset2df(dataset, columns = ['emotion','valence','arousal']):
    df = pd.DataFrame()
    for column in columns:
        df[column] = dataset[column]
    return df

train_df = dataset2df(train_dataset)
valid_df = dataset2df(valid_dataset)

total_df = pd.concat([train_df,valid_df], axis=0)
display(total_df)
# Get Centroid ... 
from collections import defaultdict

emotion_va_dict = defaultdict(dict)

for emotion_idx in range(0,7):
    valence = total_df[total_df['emotion']==emotion_idx]['valence'].mean()
    arousal = total_df[total_df['emotion']==emotion_idx]['arousal'].mean()
    # print()
    emotion_va_dict[f'{emotion_idx}_centroid'] = (valence, arousal)
emotion_va_dict

,emotion,valence,arousal
0,0,1.7,4.0
1,1,1.5,3.6
2,2,1.3,4.3
3,2,1.5,3.8
4,1,1.6,3.7
...,...,...,...
1883,5,2.3,2.1
1884,5,2.5,2.3
1885,5,2.1,1.8
1886,4,3.9,3.2


defaultdict(dict,
            {'0_centroid': (2.637565, 3.637669),
             '1_centroid': (1.7393138, 3.6382587),
             '2_centroid': (1.8028255, 3.8001664),
             '3_centroid': (2.971001, 2.981144),
             '4_centroid': (4.3137026, 3.8631923),
             '5_centroid': (1.895839, 2.6122148),
             '6_centroid': (2.205371, 3.1242967)})

In [244]:
# get centroid 
import logging
import hydra
import pandas as pd


import torch.nn as  nn 

from torch.utils.data import DataLoader
from datasets import load_from_disk

import erc

train_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy20_train4")
valid_dataset = load_from_disk("/home/hoesungryu/etri-erc/kemdy20_valid4")


def dataset2df(dataset, columns = ['emotion','valence','arousal']):
    df = pd.DataFrame()
    for column in columns:
        df[column] = dataset[column]
    return df

train_df = dataset2df(train_dataset)
valid_df = dataset2df(valid_dataset)

total_df = pd.concat([train_df,valid_df], axis=0)
display(total_df)
# Get Centroid ... 
from collections import defaultdict

emotion_va_dict = defaultdict(dict)

for emotion_idx in range(0,7):
    valence = total_df[total_df['emotion']==emotion_idx]['valence'].mean()
    arousal = total_df[total_df['emotion']==emotion_idx]['arousal'].mean()
    # print()
    emotion_va_dict[f'{emotion_idx}_centroid'] = (valence, arousal)
emotion_va_dict

,emotion,valence,arousal
0,3,3.4,2.9
1,4,4.1,3.4
2,3,3.1,2.8
3,3,3.1,3.1
4,3,3.4,3.4
...,...,...,...
2785,3,3.0,3.0
2786,0,2.7,3.5
2787,3,2.9,3.5
2788,3,2.3,3.5


defaultdict(dict,
            {'0_centroid': (3.0942307, 3.380769),
             '1_centroid': (2.3906977, 3.0767443),
             '2_centroid': (2.2208333, 3.5694444),
             '3_centroid': (2.9960432, 3.1912591),
             '4_centroid': (3.838546, 3.6015217),
             '5_centroid': (2.5082645, 2.918182),
             '6_centroid': (2.2032788, 3.2934425)})

In [ ]:
# transformation 

import erc

validation_fold: int = 4
PRETRAINED_DATA_PATH: str = "./aihub"
mode: str = "train"
wav_processor: str = "kresnik/wav2vec2-large-xlsr-korean"
sampling_rate: int = 16_000
wav_max_length: int = 112_000 # 16_000 * 7, 7secs duration
txt_processor: str = "klue/bert-base"
txt_max_length: int = 64
multilabel: bool = True
load_from_cache_file: bool = True
num_proc: int = 8
batched: bool = True
batch_size: int = 1000 # Not a torch batch_size
writer_batch_size: int = 1000
num_data: int = None
preprocess: bool = True


ds_kwargs = dict(
    # Note for hard-coded kwargs
    # generate_csv=False,
    return_bio=False,
    tokenizer_name=None,
    max_length_wav=wav_max_length,
    max_length_txt=txt_max_length,
    multilabel=multilabel,
    validation_fold=validation_fold,
    mode=mode,
    num_data=num_data,
    # PRETRAINED_DATA_PATH=PRETRAINED_DATA_PATH,
)

ds = erc.datasets.KEMDDataset(**ds_kwargs)

In [238]:
from erc.constants import emotion_va_19_dict, emotion_va_20_dict, emotion_va_19_20_dict

# paths from HF_KEMD __init__ paths

if paths =='kemdy19':
    emotion_va_dict = emotion_va_19_dict
elif paths == 'kemdy20':
    emotion_va_dict = emotion_va_19_dict
elif paths == 'kemdy19-kemdy20':
    emotion_va_dict = emotion_va_19_20_dict

def find_deuce_label(regress : torch.tensor, deuce_mask : torch.tensor) -> torch.tensor:
    '''
    Summary:
        동률인 것 중 거리가 가장 작은 것을 사용하여 라벨을 확정한다. 
    Input
       regress = torch.tensor([valence, arousal])
       deuce_maks = tensor([False,  True, False,  True, False, False, False])
    Output
        return minmum index in deuce_mask
    '''
    
    deuce_mask = deuce_mask.nonzero().squeeze().numpy()
    total_dist = []
    for index in deuce_mask:
        item = emotion_va_dict[f'{index}_centroid']
        tmp_va_tensor = torch.tensor(item)
        
        tmp_dist = torch.norm(regress-tmp_va_tensor, 2, dim=0)
        total_dist.append(tmp_dist)

    total_dist = torch.stack(total_dist, -1) # reshape -1 
    return torch.tensor(deuce_mask[total_dist.argmin()]) # select minumum value index


In [239]:
for batch in ds:
    tmp_emotion = torch.tensor(batch['emotion'].astype(float))
    deuce_mask = (tmp_emotion == tmp_emotion.max())
    if len(tmp_emotion[deuce_mask]) > 1:
        regress = torch.stack([batch['valence'], batch["arousal"]])
        gt_emotion = find_deuce_label(regress, deuce_mask)

    else:

        gt_emotion = tmp_emotion.argmax()


max 두개 이상 
tensor([0.0000, 0.4000, 0.2000, 0.4000, 0.0000, 0.0000, 0.0000],
       dtype=torch.float64)
tensor(1)


In [201]:
deuce_mask

tensor([False,  True, False,  True, False, False, False])

In [241]:
torch.ones(7,).argmax()

tensor(0)

In [242]:
torch.tensor([0.0000, 0.4000, 0.2000, 0.4000, 0.0000, 0.0000, 0.0000]).argmax()

tensor(1)

In [231]:
deuce_mask == True
for index in deuce_mask.nonzero().squeeze().numpy():
    print(index)

1
3


tensor([False,  True, False,  True, False, False, False])

<function defaultdict.values>

In [226]:
emotion_va_dict = {
    '0_centroid': (2.7013428, 3.6017902),
    '1_centroid': (1.8056872, 3.5810425),
    '2_centroid': (1.8337096, 3.7831194),
    '3_centroid': (2.989782, 3.1387265),
    '4_centroid': (4.093699, 3.7420354),
    '5_centroid': (1.9814088, 2.6549654),
    '6_centroid': (2.2050884, 3.1471236)}

In [230]:
emotion_va_dict[1]

KeyError: 1

In [61]:
import torch
cls_logits   = torch.randn([3, 9]) # Batch, Seq
cls_labels  = torch.ones([3, 9]) # Batch, Seq

emotion = torch.ones(1)
gt_arousal = torch.tensor([[3.],[2.0],[3.1]])# labels["arousal"]
gt_valence = torch.tensor([[3.],[2.0],[3.1]])# labels["valence"]

print('pred:\n',cls_logits)
print('gt:\n',cls_labels)

pred:
 tensor([[ 1.5095, -0.1732,  0.7325,  0.7860, -0.3720,  1.2313,  0.4379,  0.6280,
          0.9031],
        [ 0.9350,  0.6143,  1.5245,  0.6916,  1.9839, -1.8993,  0.2561, -0.2878,
          0.4371],
        [ 1.0684,  0.5397, -0.1996,  1.1719,  1.3606,  0.0775,  0.5241, -0.8296,
         -1.0481]])
gt:
 tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [17]:
elif cls_labels.ndim == 2:
    cls_loss = self.criterions["cls"](cls_logits, cls_labels.float())

    print(cls_labels.max(dim=1).values)
    if len(cls_labels.max(dim=1).values) > 1:
        # Compare distance with Centroid    
    else:
        # 
        cls_labels = cls_labels.argmax(dim=1)

    
print(len(cls_labels.max(dim=1).values))

tensor([1., 1., 1.])
3


In [19]:
cls_labels  = torch.ones([3, 9]) # Batch, Seq
if len(cls_labels.max(dim=1).values) > 1:
    

tensor([1., 1., 1.])


In [18]:
cls_labels  = torch.randn([3, 9]) # Batch, Seq
cls_labels.argmax(dim=1)

tensor([0, 5, 4])

INFO:erc.datasets:Instantiate train Dataset


max 두개 이상 


In [189]:
gt_emotion

tensor(6)

In [187]:
tmp_emotion

tensor([0.0000, 0.4000, 0.2000, 0.4000, 0.0000, 0.0000, 0.0000],
       dtype=torch.float64)

In [188]:
regress

tensor([2.2000, 3.1000])

In [172]:
tmp_emotion

tensor([0.0000, 0.0000, 0.0000, 0.9000, 0.1000, 0.0000, 0.0000],
       dtype=torch.float64)

In [171]:
regress

tensor([1.7000, 4.0000])

In [185]:
tmp_emotion = torch.tensor([3,1,3,1,0,2,0])
emotion_max = tmp_emotion.max()

# max 값이 두개 이상이면 그때 거리로 구한다. 
if len(tmp_emotion[tmp_emotion == tmp_emotion.max()]) > 1:
    



2

In [ ]:
ds_kwargs = dict(
    # Note for hard-coded kwargs
    # generate_csv=False,
    return_bio=False,
    tokenizer_name=None,
    max_length_wav=wav_max_length,
    max_length_txt=txt_max_length,
    multilabel=multilabel,
    validation_fold=validation_fold,
    mode="valid",
    num_data=num_data,
    # PRETRAINED_DATA_PATH=PRETRAINED_DATA_PATH,
)

ds = erc.datasets.KEMDDataset(**ds_kwargs)

In [ ]:
for batch in ds:
    tmp_emotion = torch.tensor(batch['emotion'].astype(float))
    if len(tmp_emotion.unique()) > 1:
        continue
        gt_emotion = tmp_emotion.argmax()
    else:
        regress = torch.stack([batch['valence'], batch["arousal"]])
        gt_emotion = find_duece_label(regress)
        break

In [173]:
gt_emotion

NameError: name 'gt_emotion' is not defined

In [167]:
if len(tmp_emotion.unique()) > 1:
    gt_emotion = tmp_emotion.argmax()
else:
    gt_emotion = find_duece_label(tmp_emotion)

tensor(0)


In [161]:
tmp_emotion.shape

torch.Size([7])

In [163]:
print(tmp_emotion.unique())
print(len(tmp_emotion.unique()))

tensor([0.0000, 0.1000, 0.9000], dtype=torch.float64)
3


In [147]:
tmp_emotion.max()

0.9

In [154]:
cls_labels  = torch.ones(7,) # Batch, Seq
cls_labels

tensor([1., 1., 1., 1., 1., 1., 1.])

In [158]:
cls_labels.unique()

tensor([1.])

In [ ]:
    
print(len(cls_labels.max(dim=1).values))

In [ ]:
def check

In [110]:
for batch in ds:
    regress = torch.stack([batch['valence'], batch["arousal"]])
    print(regress)
    break

tensor([1.7000, 4.0000])


tensor([1.0776, 0.4321, 0.2548, 1.5509, 2.4076, 1.3742, 0.9912])
tensor(2)


In [124]:
total_dist

tensor([1.0776, 0.4321, 0.2548, 1.5509, 2.4076, 1.3742, 0.9912])

In [121]:
total_dist = torch.stack(total_dist, -1)

In [122]:
total_dist

tensor([1.0776, 0.4321, 0.2548, 1.5509, 2.4076, 1.3742, 0.9912])

In [ ]:
outx = []
for i in range(5):
    tmp = net(x) # this will return a 10x10 tensor
    outx.append(tmp)

outx = torch.stack(outx, 2)

In [113]:
tmp_dist

tensor(1.0776)

In [99]:
regress = torch.tensor([[ds[0]['valence'], ds[0]['arousal']],
                       [ds[1]['valence'], ds[1]['arousal']],
                       [ds[2]['valence'], ds[2]['arousal']],
                    ])

In [100]:
regress

tensor([[1.7000, 4.0000],
        [1.5000, 3.6000],
        [1.3000, 4.3000]])

In [101]:
regress.shape

torch.Size([3, 2])

In [94]:
torch.norm(regress-regress, 2, dim=1)

tensor([0., 0., 0.])

In [98]:
tmp_va_tensor = torch.tensor((1.9814088, 2.6549654))
batch_size = 3
tmp_va_tensor.repeat(batch_size,1)

tensor([[1.9814, 2.6550],
        [1.9814, 2.6550],
        [1.9814, 2.6550]])

In [102]:
batch_size, *_ = regress.shape

total_dist = []
total_dist = torch.tensor()
for key, item in emotion_va_dict.items():
    tmp_va_tensor = torch.tensor(item).repeat(batch_size,1)
    
    tmp_dist = torch.norm(regress-tmp_va_tensor, 2, dim=1)
    total_dist.append(tmp_dist)
    
    


In [105]:
total_dist


[tensor([1.0776, 1.2013, 1.5656]),
 tensor([0.4321, 0.3063, 0.8790]),
 tensor([0.2548, 0.3807, 0.7430]),
 tensor([1.5509, 1.5596, 2.0503]),
 tensor([2.4076, 2.5976, 2.8489]),
 tensor([1.3742, 1.0606, 1.7806]),
 tensor([0.9912, 0.8380, 1.4657])]

In [82]:
torch.norm(regress, tmp_va_tensor)

TypeError: norm() received an invalid combination of arguments - got (Tensor, Tensor, keepdim=bool, dim=list), but expected one of:
 * (Tensor input, Number p)
 * (Tensor input, Number p, *, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: keepdim, dim
 * (Tensor input, Number p, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, Number p, tuple of ints dim, bool keepdim, *, Tensor out)
 * (Tensor input, Number p, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, Number p, tuple of names dim, bool keepdim, *, Tensor out)


In [71]:
labels = {}
for batch in ds:
    torch.stack([batch["valence"], batch["arousal"]], dim=1).float()
    break

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

## Load weight with Torchlightning


In [1]:
from erc.trainer import ERCModule
from erc.model import MLP_Mixer

WEIGHTS_PATH = '/home/hoesungryu/etri-erc/weights_AI_HUB/26908-valid_acc0.994.ckpt'

erc.trainer.train(config)
model = ERCModule(MLP_Mixer).load_from_checkpoint(WEIGHTS_PATH)
print(model.learning_rate)

TypeError: ERCModule.__init__() missing 3 required positional arguments: 'train_loader', 'valid_loader', and 'optimizer'

## Change loss function

```python
erc.optims.multi_class_focal_loss import FocalLoss
```

